<a href="https://colab.research.google.com/github/Phatdat01/recommendation/blob/dev/temp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install mlflow
!pip install gdown
!pip install pyspark
!gdown 1ZrayeVD2iZgfgB3sIt-L9LEn4zNFGgiu

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 16.5 MB 24.0 MB/s 
     |████████████████████████████████| 209 kB 101.7 MB/s 
     |████████████████████████████████| 79 kB 9.0 MB/s 
     |████████████████████████████████| 182 kB 96.1 MB/s 
     |████████████████████████████████| 575 kB 111.1 MB/s 
     |████████████████████████████████| 147 kB 84.9 MB/s 
     |████████████████████████████████| 82 kB 573 kB/s 
     |████████████████████████████████| 78 kB 8.9 MB/s 
     |████████████████████████████████| 140 kB 103.7 MB/s 
     |████████████████████████████████| 55 kB 4.6 MB/s 
     |████████████████████████████████| 62 kB 1.9 MB/s 
     |████████████████████████████████| 62 kB 1.7 MB/s 
  Created wheel for databricks-cli: filename=databricks_cli-0.17.4-py3-none-any.whl size=142894 sha256=5ef0dcc32ed5eb26a606075ae883df57d3dad1f8061551fbebb64a693fc4db7b
  Stored in directory: /root/.cache/pip/wheel

In [2]:
!pip install --upgrade --no-cache-dir gdown
!python -m gdown.cli "https://drive.google.com/uc?id=10xqEKqtA04JSKU2rF0XZQP9X34ovtgoY&confirm=t"

# !python -m gdown.cli "https://drive.google.com/uc?id=1ZrayeVD2iZgfgB3sIt-L9LEn4zNFGgiu&confirm=t"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Attempting uninstall: gdown
    Found existing installation: gdown 4.4.0
    Uninstalling gdown-4.4.0:
      Successfully uninstalled gdown-4.4.0
Downloading...
From: https://drive.google.com/uc?id=10xqEKqtA04JSKU2rF0XZQP9X34ovtgoY&confirm=t
To: /content/yelp_dataset.tar
100% 4.34G/4.34G [00:54<00:00, 79.2MB/s]


In [3]:
import os
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS
from pyspark.ml.recommendation import ALSModel
from pyspark.sql import Row
import numpy as np
from datetime import datetime
import tarfile
import matplotlib.pyplot as plt
import pandas as pd
from pyspark.sql import Window
from pyspark.sql.window import *

In [6]:
# %cd /content
# # !mkdir yelp_dataset
# my_tar = tarfile.open('/content/yelp_dataset.tar')
# my_tar.extract('yelp_academic_dataset_review.json',".")
# my_tar.close()
# !rm /content/yelp_dataset.tar

/content


In [4]:
spark = SparkSession.builder\
        .master("local")\
        .appName("Colab")\
        .config('spark.ui.port', '5000')\
        .getOrCreate()

In [11]:
df = spark.read.json("/content/yelp_academic_dataset_review.json")
df.printSchema()
df.show(5)

root
 |-- business_id: string (nullable = true)
 |-- cool: long (nullable = true)
 |-- date: string (nullable = true)
 |-- funny: long (nullable = true)
 |-- review_id: string (nullable = true)
 |-- stars: double (nullable = true)
 |-- text: string (nullable = true)
 |-- useful: long (nullable = true)
 |-- user_id: string (nullable = true)

+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|         business_id|cool|               date|funny|           review_id|stars|                text|useful|             user_id|
+--------------------+----+-------------------+-----+--------------------+-----+--------------------+------+--------------------+
|XQfwVwDr-v0ZS3_Cb...|   0|2018-07-07 22:09:11|    0|KU_O5udG6zpxOg-Vc...|  3.0|If you decide to ...|     0|mh_-eMZ6K5RLWhZyI...|
|7ATYjTIgM3jUlt4UM...|   1|2012-01-03 15:28:18|    0|BiTunyQ73aT9WBnpR...|  5.0|I've taken a lot ...|     1|OyoGAe7OKpv6SyGZT...|
|YjUWPp

In [12]:
ratings=df.select("user_id","business_id","stars","date","cool","funny","useful")

In [13]:
userRatings=ratings.groupBy("user_id").count()
## take id type int
window = Window.orderBy(col('user_id'))
userRatings = userRatings.withColumn('userid', row_number().over(window))

##Because ALS in spark require users or items is  in int type, we must process it

## if you use this, for the large data, it processes wrong
# userRatings=ratings.groupBy("user_id").count().withColumn("userid", monotonically_increasing_id().cast(IntegerType()))
userRatings.show(5)

+--------------------+-----+------+
|             user_id|count|userid|
+--------------------+-----+------+
|---1lKK3aKOuomHnw...|    1|     1|
|---2PmXbF47D870st...|   28|     2|
|---UgP94gokyCDuB5...|   11|     3|
|---fa6ZK37T9NjkGK...|    1|     4|
|---r61b7EpVPkb4UV...|    5|     5|
+--------------------+-----+------+
only showing top 5 rows



In [14]:
buiRatings=ratings.groupBy("business_id").count()
## take id type int
window = Window.orderBy(col('business_id'))
buiRatings = buiRatings.withColumn('businessid', row_number().over(window))

# buiRatings=ratings.groupBy("business_id").count().withColumn("businessid", monotonically_increasing_id().cast(IntegerType()))
newratings=ratings.join(userRatings, ['user_id'])
newratings=newratings.join(buiRatings, ['business_id'])
newratings=newratings.select("userid","businessid","stars","date")

In [15]:
newratings=newratings.withColumn("stars",col("stars").cast(IntegerType()))
newratings=newratings.withColumn("date",to_timestamp("date"))

print(newratings)
newratings.show(5)

DataFrame[userid: int, businessid: int, stars: int, date: timestamp]
+------+----------+-----+-------------------+
|userid|businessid|stars|               date|
+------+----------+-----+-------------------+
|    20|     40541|    1|2015-05-03 03:53:01|
|    20|     76452|    5|2015-05-21 03:18:49|
|    26|      4489|    4|2020-10-29 23:34:47|
|    36|    105552|    5|2019-04-18 05:17:58|
|    36|     29020|    2|2017-02-03 06:14:50|
+------+----------+-----+-------------------+
only showing top 5 rows



# New

In [ ]:
num=int(newratings.count()*0.3)
ortherDate=newratings.orderBy(col("date").desc())
test=ortherDate.limit(int(num))
train=ortherDate.subtract(test)

In [ ]:
test.orderBy(col("date").asc()).show(10)
train.orderBy(col("date").desc()).show(10)

+-------+----------+-----+-------------------+
| userid|businessid|stars|               date|
+-------+----------+-----+-------------------+
|1832830|     31615|    1|2019-01-06 03:47:03|
|1042942|     86890|    4|2019-01-06 03:47:07|
|1781463|     79615|    5|2019-01-06 03:47:19|
|1213934|     58044|    1|2019-01-06 03:47:33|
| 348358|     17893|    1|2019-01-06 03:47:45|
| 842914|     57367|    1|2019-01-06 03:48:04|
|1325386|     80502|    1|2019-01-06 03:48:57|
|1310940|     67071|    5|2019-01-06 03:49:10|
|1597092|     79182|    2|2019-01-06 03:51:07|
| 834344|     94428|    5|2019-01-06 03:51:50|
+-------+----------+-----+-------------------+
only showing top 10 rows

+-------+----------+-----+-------------------+
| userid|businessid|stars|               date|
+-------+----------+-----+-------------------+
|1369796|     79423|    4|2019-01-06 03:46:45|
|1583891|    131428|    5|2019-01-06 03:45:40|
|1406586|     42322|    4|2019-01-06 03:45:18|
| 520738|    123025|    2|2019-01-

In [ ]:
userTrain=train.groupBy('userid').count().select('userid')
## remove same id 
duplicate=test.join(userTrain,['userid'])
new=test.subtract(duplicate)

In [ ]:
# def SplitTest(source,data):
#     train,test=data.randomSplit([0.3,0.7])
#     try:
#         return source.union(train)
#     except:
#         return train

In [ ]:
# ## Keep data test with keep
# newTrain = None
# userNew=new.groupBy('userid').count().select('userid')
# lstuserNew=userNew.rdd.flatMap(lambda x: x).collect()

# for i in lstuserNew:
#     userI=new.filter(col('userid')==i)
#     newTrain=SplitTest(newTrain,userI)
    

In [ ]:
## remove test data to union to train and for test
newTrain,_=new.randomSplit([0.5,0.5])
numNewTrain=newTrain.count()
print(numNewTrain)
descDate=train.orderBy(col("date").desc())
forTest=descDate.limit(numNewTrain)

newtrain=train.subtract(forTest).union(newTrain)
newTest=test.subtract(newTrain).union(forTest)

535272


In [ ]:
newTest.show(20)

+------+----------+-----+-------------------+
|userid|businessid|stars|               date|
+------+----------+-----+-------------------+
|   154|     52768|    3|2021-12-02 03:52:28|
|   192|     13759|    4|2022-01-16 16:37:34|
|   669|     35238|    1|2021-12-23 02:40:10|
|   828|     20133|    4|2019-03-16 00:27:45|
|  2496|     67586|    1|2020-11-09 14:16:18|
|  2688|     64745|    5|2021-02-28 03:00:23|
|  2803|    107602|    5|2019-09-02 22:54:48|
|  2816|     11831|    4|2019-08-06 04:33:08|
|  3275|       659|    5|2019-08-27 20:06:01|
|  3505|     83614|    5|2019-04-10 03:15:51|
|  3706|     81675|    5|2021-07-17 21:51:21|
|  3767|     59864|    1|2019-05-20 03:36:53|
|  4053|    108136|    1|2021-11-11 18:20:22|
|  4072|    120113|    5|2020-11-25 08:15:11|
|  4127|     35692|    1|2020-08-26 19:06:27|
|  4866|     32504|    5|2019-02-02 02:37:18|
|  5198|     61557|    5|2019-04-03 23:34:15|
|  5857|    130940|    5|2021-08-22 23:15:07|
|  6237|     62947|    1|2020-01-0

In [ ]:
val,tes=newTest.randomSplit([2/3,1/3])
print(f"num newtest: {newTest}")
print(f"num val: {val}")
print(f"num tes: {tes}")

num newtest: DataFrame[userid: int, businessid: int, stars: int, date: timestamp]
num val: DataFrame[userid: int, businessid: int, stars: int, date: timestamp]
num tes: DataFrame[userid: int, businessid: int, stars: int, date: timestamp]


In [ ]:
als= ALS(rank=4, maxIter=4, regParam=0.35, userCol="userid", itemCol="businessid", ratingCol="stars", nonnegative=True,implicitPrefs = False,coldStartStrategy="drop")
model = als.fit(newtrain)
predictions=model.transform(newtrain)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"training RMSE={rmse}")

predictions=model.transform(val)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"validate RMSE={rmse}")

In [ ]:
predictions=model.transform(tes)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars", predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print(f"validate RMSE={rmse}")

validate RMSE=1.5808870760015483


# Choose

In [7]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [16]:
def trainModel(train,test,rank,iter,reg):
  als= ALS(rank=rank, maxIter=iter, regParam=reg, userCol="userid", itemCol="businessid", ratingCol="stars", nonnegative=True,implicitPrefs = False,coldStartStrategy="drop")
  model = als.fit(train)
  predictions=model.transform(test)
  evaluator = RegressionEvaluator(metricName="rmse", labelCol="stars", predictionCol="prediction")
  rmse = evaluator.evaluate(predictions)
  print(f"rank: {rank},iter: {iter},reg: {reg}, RMSE={rmse}")
  return predictions,rmse

def SaveRow(rank,iter,reg,rmse):
  f=open("/content/drive/MyDrive/TLCN-19133035-19133018/score2.txt", "a")
  f.write(f"\n{rank},{iter},{reg},{rmse}")
  f.close()

In [ ]:
num=int(newratings.count()*0.1)
ortherDate=newratings.orderBy(col("date").desc())
testing=ortherDate.limit(int(num))
train=ortherDate.subtract(testing)

ortherDate=train.orderBy(col("date").desc())
validate=ortherDate.limit(int(num)*2)
training=train.subtract(validate)
 
print(f"training: {training.count()}")
print(f"validate: {validate.count()}")
print(f"testing: {testing.count()}")

ranks=[3,5,10,15,20,25]
iters=[3,5,10,15,20,25]
regs=[0.1,0.2,0.3,0.4,0.5]

for i in ranks:
  model,rmse=trainModel(training,validate,i,iters[0],regs[0])
  SaveRow(i,iters[0],regs[0],rmse)

for i in iters:
  model,rmse=trainModel(training,validate,ranks[0],i,regs[0])
  SaveRow(ranks[0],i,regs[0],rmse)

for i in iters:
  model,rmse=trainModel(training,validate,ranks[0],iters[0],i)
  SaveRow(ranks[0],iters[0],i,rmse)

training: 4893172
validate: 1398056
testing: 699028
rank: 3,iter: 3,reg: 0.1, RMSE=1.7515949072299313
rank: 5,iter: 3,reg: 0.1, RMSE=1.817529796409866
